<a href="https://colab.research.google.com/github/ARADHYA299/GoogleSolutionChallenge2025/blob/main/UPDATED_TUTOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q Gradio
!pip install -q transformers
!pip install -q yfinance
!pip install -q nltk

In [5]:
import gradio as gr
import requests
import time
import os
import re
import numpy as np
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd
import plotly.graph_objects as go
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk

# Download NLTK resources
try:
    nltk.data.find('vader_lexicon')
except LookupError:
    nltk.download('vader_lexicon')

# Get API key from Hugging Face Spaces secret
HF_API_KEY = os.environ.get("tutorhfkey")
if not HF_API_KEY:
    print("Warning: API key 'tutorhfkey' not found in environment variables")
    # Fallback only for development
    HF_API_KEY = "YOUR_API_KEY_PLACEHOLDER"

# API URLs
HF_API_URL = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.2"
NEWS_API_KEY = os.environ.get("newsapikey", "38666Z58XHGQB9X6")
NEWS_API_URL = "https://newsapi.org/v2/everything"
headers = {"Authorization": f"Bearer {HF_API_KEY}"}

# Initialize sentiment analyzer
sentiments = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
finance_sentiment = SentimentIntensityAnalyzer()

# System instructions for the financial tutor
SYSTEM_INSTRUCTIONS = """You are a financial markets tutor and adviser designed to educate and recommend/advise beginners, intermediates or experts about investing, financial instruments, money markets and market dynamics.
Answer briefly and to the point, don't explain unnecessarily, and use things like real life examples or the things with which they can relate them.

Capabilities:
- Teach financial concepts in an interactive and engaging way.
- Guide users on different types of financial markets (stocks, bonds, crypto, commodities, etc.).
- Explain investment strategies, risk management, and portfolio diversification but tell these only when asked.
- Answer questions related to fundamental and technical analysis.
- If they give you certain budgets provide them the best areas to invest along with risk involved but warn them about the risks and also advise to think before investing.
- You can analyze user sentiment to provide personalized financial advice.
- You can explain stock trends and market news sentiment when requested.

Guidelines:
- Begin by understanding the user's financial knowledge level.
- If the user is new, first ask them which financial market they are interested in.
- Provide structured, step-by-step explanations with examples.
- Use simple language and try to give response in an example or things with which the user can relate.
- If the user seems anxious or negative about investing, provide reassuring advice about risk management.
- If the user seems overconfident, remind them about market volatility and risk assessment."""

# Function to query the model with retry logic
def query_model(prompt, max_retries=3):
    """Send a request to the Hugging Face API with retry logic"""
    payload = {
        "inputs": prompt,
        "parameters": {
            "max_new_tokens": 512,
            "temperature": 0.7,
            "top_p": 0.95
        }
    }

    for attempt in range(max_retries):
        try:
            response = requests.post(
                HF_API_URL,
                headers=headers,
                json=payload,
                timeout=30
            )

            # Handle successful response
            if response.status_code == 200:
                return response.json()[0]["generated_text"]

            # Handle model loading
            elif response.status_code == 503:
                wait_time = 2 * (attempt + 1)
                print(f"Model is loading. Waiting {wait_time} seconds...")
                time.sleep(wait_time)

                # If last attempt, return a friendly message
                if attempt == max_retries - 1:
                    return "The model is currently loading. Please try again in a moment."

            # Handle other errors
            else:
                print(f"Error: Received status code {response.status_code}")
                error_msg = f"Error: Received status code {response.status_code}. "
                if attempt == max_retries - 1:
                    return f"{error_msg} Please try again later."
                time.sleep(2)

        except Exception as e:
            print(f"Exception occurred: {str(e)}")
            if attempt == max_retries - 1:
                return f"Sorry, I encountered an error: {str(e)}. Please try again later."
            time.sleep(2)

    return "I'm having trouble connecting to the server. Please try again later."

# Format the conversation for the model
def format_prompt(messages):
    """Format the conversation history for the Mistral model"""
    prompt = f"<s>[INST] {SYSTEM_INSTRUCTIONS}\n\n"

    for i, message in enumerate(messages):
        role = message["role"]
        content = message["content"]

        if role == "user":
            if i == len(messages) - 1:
                # Last user message
                prompt += f"{content} [/INST]"
            else:
                # User message followed by assistant message
                assistant_msg = messages[i+1]["content"] if i+1 < len(messages) and messages[i+1]["role"] == "assistant" else ""
                prompt += f"{content} [/INST] {assistant_msg} </s>"
                if i+2 < len(messages) and messages[i+2]["role"] == "user":
                    prompt += "<s>[INST] "

    return prompt

# Extract stock ticker symbols from user message
def extract_tickers(message):
    # Look for standard stock ticker patterns (1-5 uppercase letters)
    ticker_pattern = r'\b[A-Z]{1,5}\b'
    potential_tickers = re.findall(ticker_pattern, message)

    # Filter out common English words and acronyms that aren't tickers
    common_words = {'I', 'A', 'AN', 'THE', 'AND', 'OR', 'IF', 'IS', 'IT', 'BE', 'TO', 'IN', 'ON', 'AT', 'OF', 'FOR'}
    filtered_tickers = [ticker for ticker in potential_tickers if ticker not in common_words]

    return filtered_tickers

# Get news sentiment for a specific topic or ticker
def get_news_sentiment(query, max_results=5):
    if not NEWS_API_KEY or NEWS_API_KEY == "38666Z58XHGQB9X6":
        return "News API key not configured. Unable to analyze news sentiment."

    try:
        params = {
            'q': query,
            'language': 'en',
            'sortBy': 'publishedAt',
            'pageSize': max_results,
            'apiKey': NEWS_API_KEY
        }

        response = requests.get(NEWS_API_URL, params=params)

        if response.status_code != 200:
            return f"Error fetching news: {response.status_code}"

        articles = response.json().get('articles', [])

        if not articles:
            return f"No recent news found for {query}"

        # Analyze sentiment of headlines and descriptions
        results = []
        for article in articles:
            headline = article.get('title', '')
            description = article.get('description', '')
            text = headline + ' ' + description

            # Use VADER for financial sentiment analysis
            sentiment_score = finance_sentiment.polarity_scores(text)
            compound_score = sentiment_score['compound']

            if compound_score >= 0.05:
                sentiment = "POSITIVE"
            elif compound_score <= -0.05:
                sentiment = "NEGATIVE"
            else:
                sentiment = "NEUTRAL"

            results.append({
                'headline': headline,
                'sentiment': sentiment,
                'score': compound_score,
                'source': article.get('source', {}).get('name', 'Unknown'),
                'url': article.get('url', '')
            })

        # Calculate overall sentiment
        avg_score = sum(r['score'] for r in results) / len(results)

        if avg_score >= 0.05:
            overall = "POSITIVE"
        elif avg_score <= -0.05:
            overall = "NEGATIVE"
        else:
            overall = "NEUTRAL"

        return {
            'overall_sentiment': overall,
            'overall_score': avg_score,
            'articles': results[:3]  # Return only top 3 articles
        }

    except Exception as e:
        return f"Error analyzing news sentiment: {str(e)}"

# Get stock data and perform technical analysis
def get_stock_data(ticker, period='1mo'):
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(period=period)

        if hist.empty:
            return f"No data found for ticker {ticker}"

        # Calculate some basic technical indicators
        hist['SMA20'] = hist['Close'].rolling(window=20).mean()
        hist['SMA50'] = hist['Close'].rolling(window=50).mean()

        # Determine trend
        current_price = hist['Close'].iloc[-1]
        week_ago_price = hist['Close'].iloc[-6] if len(hist) >= 6 else hist['Close'].iloc[0]
        month_ago_price = hist['Close'].iloc[0]

        weekly_change = ((current_price - week_ago_price) / week_ago_price) * 100
        monthly_change = ((current_price - month_ago_price) / month_ago_price) * 100

        # Determine if stock is above or below moving averages
        above_sma20 = current_price > hist['SMA20'].iloc[-1] if not np.isnan(hist['SMA20'].iloc[-1]) else "Unknown"
        above_sma50 = current_price > hist['SMA50'].iloc[-1] if not np.isnan(hist['SMA50'].iloc[-1]) else "Unknown"

        # Calculate volatility (standard deviation of returns)
        hist['Daily_Return'] = hist['Close'].pct_change()
        volatility = hist['Daily_Return'].std() * 100  # Convert to percentage

        # Prepare chart data
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=hist.index, y=hist['Close'], mode='lines', name='Price'))
        if not hist['SMA20'].isnull().all():
            fig.add_trace(go.Scatter(x=hist.index, y=hist['SMA20'], mode='lines', name='SMA20'))
        if not hist['SMA50'].isnull().all():
            fig.add_trace(go.Scatter(x=hist.index, y=hist['SMA50'], mode='lines', name='SMA50'))

        fig.update_layout(
            title=f"{ticker} Price Chart",
            xaxis_title="Date",
            yaxis_title="Price",
            height=500,
            template="plotly_white"
        )

        return {
            'ticker': ticker,
            'current_price': current_price,
            'weekly_change': weekly_change,
            'monthly_change': monthly_change,
            'above_sma20': above_sma20,
            'above_sma50': above_sma50,
            'volatility': volatility,
            'chart': fig
        }

    except Exception as e:
        return f"Error analyzing {ticker}: {str(e)}"

# Analyze user message sentiment
def analyze_user_sentiment(message):
    try:
        # Use sentiment analysis pipeline
        result = sentiments(message)
        if not result:
            return None

        sentiment_result = result[0]
        sentiment_label = sentiment_result['label']
        confidence = sentiment_result['score']

        # Use NLTK VADER for more nuanced analysis
        vader_scores = finance_sentiment.polarity_scores(message)

        return {
            'sentiment': sentiment_label,
            'confidence': confidence,
            'compound_score': vader_scores['compound'],
            'positive': vader_scores['pos'],
            'negative': vader_scores['neg'],
            'neutral': vader_scores['neu']
        }
    except Exception as e:
        print(f"Error in sentiment analysis: {str(e)}")
        return None

# Extract investment budget from message
def extract_budget(message):
    # Pattern for currency amounts
    budget_pattern = r'(\$|£|€|₹)?\s?(\d+[,.]?\d*)\s?(thousand|million|billion|k|m|b)?\s?(dollars|usd|rupees|inr|euros|eur|pounds|gbp)?'

    match = re.search(budget_pattern, message.lower())
    if match:
        amount = match.group(2).replace(',', '')
        multiplier = match.group(3) if match.group(3) else ''

        try:
            amount = float(amount)

            if multiplier:
                if multiplier.lower() in ['k', 'thousand']:
                    amount *= 1000
                elif multiplier.lower() in ['m', 'million']:
                    amount *= 1000000
                elif multiplier.lower() in ['b', 'billion']:
                    amount *= 1000000000

            currency = match.group(1) if match.group(1) else (match.group(4) if match.group(4) else 'USD')

            return {
                'amount': amount,
                'currency': currency
            }
        except:
            pass

    return None

# Extract risk tolerance from user message
def extract_risk_tolerance(message):
    msg = message.lower()

    risk_phrases = {
        'very low': ['very conservative', 'extremely safe', 'no risk', 'safest possible', 'can\'t afford to lose'],
        'low': ['conservative', 'safe', 'low risk', 'minimal risk', 'safety', 'secure'],
        'moderate': ['balanced', 'moderate', 'middle ground', 'medium risk', 'some risk'],
        'high': ['aggressive', 'high risk', 'risky', 'growth focused', 'willing to take risks'],
        'very high': ['very aggressive', 'highest risk', 'extremely risky', 'maximum growth', 'speculative']
    }

    for tolerance, phrases in risk_phrases.items():
        if any(phrase in msg for phrase in phrases):
            return tolerance

    return None

# Identify financial education level
def identify_education_level(message):
    msg = message.lower()

    if any(term in msg for term in ['beginner', 'new', 'novice', 'starting', 'basics', 'fundamental', 'never invested']):
        return 'beginner'
    elif any(term in msg for term in ['intermediate', 'some experience', 'familiar', 'understand']):
        return 'intermediate'
    elif any(term in msg for term in ['advanced', 'expert', 'professional', 'experienced', 'seasoned']):
        return 'advanced'

    return None

# Enhanced chatbot logic with sentiment analysis
def respond(message, chat_history, conversation_state, user_profile):
    """Process user message and get response from the model"""

    # Initialize conversation if empty
    if not conversation_state:
        conversation_state = [
            {"role": "assistant", "content": "Hello! I'm your financial markets tutor. Before we begin, could you tell me your current level of knowledge about financial markets? Are you a beginner, intermediate, or advanced?"}

            ]

    # Analyze user message
    sentiment_data = analyze_user_sentiment(message)
    education_level = identify_education_level(message)
    risk_tolerance = extract_risk_tolerance(message)
    budget = extract_budget(message)
    tickers = extract_tickers(message)

    # Update user profile
    if education_level:
        user_profile["education_level"] = education_level
    if risk_tolerance:
        user_profile["risk_tolerance"] = risk_tolerance
    if budget:
        user_profile["budget"] = budget

    # Process ticker information if found
    stock_data = None
    news_sentiment = None

    if tickers and (re.search(r'stock|price|ticker|trend|chart|analysis', message.lower()) or
                   any(ticker.lower() in message.lower() for ticker in tickers)):
        # Only process the first ticker for simplicity
        ticker = tickers[0]
        stock_data = get_stock_data(ticker)
        news_sentiment = get_news_sentiment(ticker)

    # Get investment recommendations if user profile has enough information
    recommendations = None
    if "risk_tolerance" in user_profile and "budget" in user_profile and re.search(r'invest|allocation|portfolio|recommend|suggest|advice', message.lower()):
        recommendations = get_investment_recommendations(user_profile["risk_tolerance"], user_profile["budget"])

    # Add user message to conversation
    conversation_state.append({"role": "user", "content": message})

    # Format the prompt with additional context and get response
    prompt = format_prompt(conversation_state, sentiment_data, stock_data, news_sentiment)
    response_text = query_model(prompt)

    # Extract just the model's response part
    if "[/INST]" in response_text:
        response_text = response_text.split("[/INST]")[-1].strip()

    # Clean up response text (remove trailing conversation markers)
    response_text = response_text.replace("</s>", "").strip()

    # Enhance response with additional data if available
    if isinstance(stock_data, dict) and 'chart' in stock_data:
        chart = stock_data['chart']
        response_text += "\n\n[CHART_PLACEHOLDER]"  # Will be replaced with actual chart

    # Add recommendations if available
    if recommendations:
        recommendation_text = f"\n\nBased on your {user_profile['risk_tolerance']} risk tolerance"
        if 'budget' in user_profile and 'amount' in user_profile['budget']:
            amount_str = f"{user_profile['budget']['amount']:,.2f}"
            recommendation_text += f" and budget of {amount_str}"
        recommendation_text += f", here's a suggested portfolio allocation:\n\n"
        recommendation_text += recommendations['description'] + "\n\n"

        # Format allocation as a table
        allocation_text = "Investment Type | Percentage"
        allocation_text += "\n--------------|----------"
        for item in recommendations['allocation']:
            allocation_text += f"\n{item['type']} | {item['percentage']}%"

        response_text += "\n\n" + recommendation_text + allocation_text

    # Add assistant response to conversation
    conversation_state.append({"role": "assistant", "content": response_text})

    # Update the chat history for Gradio display
    # If we have a chart, we need special handling
    has_chart = isinstance(stock_data, dict) and 'chart' in stock_data

    if has_chart:
        # Remove the placeholder in the UI version
        display_response = response_text.replace("[CHART_PLACEHOLDER]", "")
        chat_history.append((message, display_response))
        return chat_history, conversation_state, user_profile, stock_data['chart'] if has_chart else None
    else:
        chat_history.append((message, response_text))
        return chat_history, conversation_state, user_profile, None

# Function to create personalized financial reports
def create_financial_report(user_profile):
    """Generate a personalized financial report based on user profile"""
    if not user_profile or "education_level" not in user_profile:
        return "Not enough user data collected to generate a personalized report."

    # Create a report header
    now = datetime.now()
    report = f"# Personal Financial Report\n\n"
    report += f"Generated on: {now.strftime('%B %d, %Y')}\n\n"

    # Add user profile information
    report += "## Your Profile\n\n"
    report += f"- Knowledge Level: {user_profile.get('education_level', 'Not specified')}\n"

    if "risk_tolerance" in user_profile:
        report += f"- Risk Tolerance: {user_profile.get('risk_tolerance', 'Not specified')}\n"

    if "budget" in user_profile and "amount" in user_profile["budget"]:
        amount = user_profile["budget"]["amount"]
        currency = user_profile["budget"].get("currency", "USD")
        report += f"- Investment Budget: {amount:,.2f} {currency}\n"

    # Add recommendations section if we have risk tolerance
    if "risk_tolerance" in user_profile:
        recommendations = get_investment_recommendations(user_profile["risk_tolerance"],
                                                         user_profile.get("budget", None))

        if recommendations:
            report += "\n## Investment Recommendations\n\n"
            report += recommendations["description"] + "\n\n"

            # Create a table of allocations
            report += "| Investment Type | Percentage |"
            if "budget" in user_profile and "amount" in user_profile["budget"]:
                report += " Amount |"
            report += "\n|-----------------|------------|"
            if "budget" in user_profile and "amount" in user_profile["budget"]:
                report += "--------|"
            report += "\n"

            for item in recommendations["allocation"]:
                report += f"| {item['type']} | {item['percentage']}% |"
                if "budget" in user_profile and "amount" in user_profile["budget"]:
                    amount = item.get("amount", 0)
                    report += f" {amount:,.2f} |"
                report += "\n"

    # Add educational resources based on level
    report += "\n## Recommended Educational Resources\n\n"

    if user_profile.get("education_level") == "beginner":
        report += "### For Beginners:\n\n"
        report += "1. **Investopedia Basics** - Fundamental concepts and terminology\n"
        report += "2. **Khan Academy: Personal Finance** - Free courses on investing basics\n"
        report += "3. **'A Random Walk Down Wall Street'** by Burton Malkiel - Classic book for beginners\n"
        report += "4. **Robinhood Learn** - Simple explanations of investment concepts\n"
    elif user_profile.get("education_level") == "intermediate":
        report += "### For Intermediate Investors:\n\n"
        report += "1. **'The Intelligent Investor'** by Benjamin Graham - Value investing principles\n"
        report += "2. **Morningstar Investment Classroom** - More advanced investment concepts\n"
        report += "3. **Yahoo Finance** - Research tools and market analysis\n"
        report += "4. **'The Little Book of Common Sense Investing'** by John Bogle\n"
    else:  # advanced
        report += "### For Advanced Investors:\n\n"
        report += "1. **Bloomberg Terminal** (if accessible) - Professional-grade research\n"
        report += "2. **CFA Institute Resources** - Professional investment analysis\n"
        report += "3. **'Security Analysis'** by Benjamin Graham and David Dodd\n"
        report += "4. **Journal of Finance** - Academic research on financial markets\n"

    # Add market outlook section
    report += "\n## Current Market Outlook\n\n"
    report += "This section would typically contain current market analysis and trends.\n"
    report += "For real-time and accurate market outlook, consider consulting financial news sources like:\n\n"
    report += "- Wall Street Journal\n"
    report += "- Financial Times\n"
    report += "- Bloomberg\n"
    report += "- CNBC\n"

    # Add disclaimer
    report += "\n## Disclaimer\n\n"
    report += "_This report is generated based on your interaction with the Financial Markets Tutor. "
    report += "It is for educational purposes only and does not constitute financial advice. "
    report += "Always consult with a qualified financial advisor before making investment decisions._\n"

    return report

# Create the Gradio interface with enhanced features
def create_interface():
    with gr.Blocks(theme=gr.themes.Soft()) as demo:
        gr.Markdown("# 📊 Advanced Financial Markets Tutor & Adviser 📊")
        gr.Markdown("Ask questions about financial markets, investing strategies, and more! The tutor analyzes your sentiment and provides personalized advice.")

        with gr.Row():
            with gr.Column(scale=7):
                chatbot = gr.Chatbot(
                    value=[],
                    show_label=False,
                    height=500,
                    bubble_full_width=False,
                    show_copy_button=True,
                )

                conversation_state = gr.State([])
                user_profile = gr.State({})

                with gr.Row():
                    message = gr.Textbox(
                        show_label=False,
                        placeholder="Type your question here...",
                        scale=5
                    )
                    submit = gr.Button("Send", scale=1)

            # Right sidebar for real-time stock data and sentiment analysis
            with gr.Column(scale=3):
                with gr.Tab("Profile"):
                    gr.Markdown("### Your Financial Profile")
                    profile_info = gr.JSON(label="Profile Data", value={})

                    generate_report_btn = gr.Button("Generate Financial Report")
                    report_output = gr.Markdown(label="Personal Financial Report")

                with gr.Tab("Market Data"):
                    stock_input = gr.Textbox(label="Enter Stock Ticker", placeholder="e.g. AAPL")
                    stock_btn = gr.Button("Get Stock Data")
                    stock_plot = gr.Plot(label="Stock Price Chart")
                    stock_info = gr.JSON(label="Stock Information")

                with gr.Tab("News Sentiment"):
                    news_input = gr.Textbox(label="Topic or Ticker", placeholder="e.g. bitcoin or TSLA")
                    news_btn = gr.Button("Analyze News Sentiment")
                    news_output = gr.JSON(label="News Sentiment Analysis")

                with gr.Tab("Market Mood"):
                    gr.Markdown("### Market Sentiment Tracker")
                    market_mood = gr.Label(label="Current Market Mood")
                    update_mood_btn = gr.Button("Update Market Mood")

        # Stock chart display area
        stock_chart_output = gr.Plot(visible=False)

        # Initialize the chat with a greeting
        def initialize_chat():
            initial_message = "Hello! I'm your financial markets tutor. Before we begin, could you tell me your current level of knowledge about financial markets? Are you a beginner, intermediate, or advanced?"
            return [(None, initial_message)], [{"role": "assistant", "content": initial_message}], {}

        # Functions for handling user input
        def user_input(message, chat_history, conversation_state, user_profile):
            if message == "":
                return chat_history, conversation_state, user_profile, None

            chat_history, conversation_state, user_profile, chart = respond(message, chat_history, conversation_state, user_profile)

            # Update profile display
            profile_info.update(value=user_profile)

            # Show chart if available
            if chart:
                stock_chart_output.update(value=chart, visible=True)

            return chat_history, conversation_state, user_profile, gr.update(value="")

        # Function to get standalone stock data
        def fetch_stock_data(ticker):
            if not ticker:
                return None, {"error": "Please enter a valid ticker symbol"}

            data = get_stock_data(ticker)

            if isinstance(data, dict) and 'chart' in data:
                # Create a clean info object without the chart
                info = {k: v for k, v in data.items() if k != 'chart'}
                return data['chart'], info
            else:
                return None, {"error": str(data)}

        # Function to get news sentiment analysis
        def fetch_news_sentiment(query):
            if not query:
                return {"error": "Please enter a topic or ticker"}

            return get_news_sentiment(query)

        # Function to generate market mood
        def update_market_mood():
            # This would ideally use a more sophisticated analysis of market indicators
            # For demonstration, we'll use a simple random approach
            moods = [
                {"Market Fear": 0.7, "Market Neutral": 0.2, "Market Greed": 0.1},
                {"Market Fear": 0.3, "Market Neutral": 0.4, "Market Greed": 0.3},
                {"Market Fear": 0.1, "Market Neutral": 0.3, "Market Greed": 0.6}
            ]
            return np.random.choice(moods)

        # Function to generate financial report
        def generate_report(user_profile):
            return create_financial_report(user_profile)

        # Wire up the interface components
        message.submit(user_input, [message, chatbot, conversation_state, user_profile],
                      [chatbot, conversation_state, user_profile, message])
        submit.click(user_input, [message, chatbot, conversation_state, user_profile],
                    [chatbot, conversation_state, user_profile, message])

        # Stock data tab functionality
        stock_btn.click(fetch_stock_data, [stock_input], [stock_plot, stock_info])

        # News sentiment tab functionality
        news_btn.click(fetch_news_sentiment, [news_input], [news_output])

        # Market mood tab functionality
        update_mood_btn.click(update_market_mood, [], [market_mood])

        # Report generation functionality
        generate_report_btn.click(generate_report, [user_profile], [report_output])

        # Initialize the chat on page load
        demo.load(initialize_chat, [], [chatbot, conversation_state, user_profile])

    return demo

# Launch the app
if __name__ == "__main__":
    demo = create_interface()
    demo.launch()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Device set to use cpu
<ipython-input-5-5c162d1d8fa3>:551: UserWarning:

You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.

<ipython-input-5-5c162d1d8fa3>:551: DeprecationWarning:

The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.



Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://041725269b4a948e0f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
